<a href="https://colab.research.google.com/github/ameland/ameland/blob/master/MMP3linkedlist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
from IPython import display
import sklearn.metrics.pairwise
import scipy.spatial.distance
import warnings
import time
import pandas as pd
import math
from collections import deque

In [ ]:
#initial parameters
L = 20 #box size
Rc = 2.5 # cutoff distance
Rcl = 4
num_box_1D = int(L/Rcl)
density = 0.8
steps = 10
eps = 1
sigma = 1
m = 1
dt = 0.001

In [ ]:
#displacement vectors relationship with neighboring 13 boxes
d_vector = pd.DataFrame(np.array([[0, 0, 0], [0, 1, 0], [1, 1, 0], [1, 0, 0],[1, -1, 0], 
                                  [-1, 1, 1], [-1, 0, 1], [-1, -1, 1], [0, 1, 1], [0, 0, 1], 
                                  [0, -1, 1], [1, 1, 1], [1, 0, 1], [1, -1, 1]]),columns=['xb', 'yb', 'zb'])
d_vector.head(3)

,xb,yb,zb
0,0,0,0
1,0,1,0
2,1,1,0


In [ ]:
# 75 boxes and their bottom corner coordinates
boxes = pd.DataFrame(columns=['x_bottom', 'y_bottom', 'z_bottom','pairing_boxes_id'])
pd.set_option('display.max_rows', 10)
for i in range(int(num_box_1D**3)):
    boxes.loc[i, 'x_bottom'] = (i%num_box_1D) * Rcl 
    boxes.loc[i, 'y_bottom'] = Rcl * math.floor(i%(num_box_1D**2)/num_box_1D)
    boxes.loc[i, 'z_bottom'] = Rcl * math.floor(i/(num_box_1D**2))


In [ ]:
# copy 27 cubes (each cube contains 75 boxes) for PBCs
boxesPBC = boxes.copy()
for x in range(-1, 2):
    for y in range(-1, 2):
        for z in range(-1, 2):
            if (x==0) & (y==0) & (z==0):
                continue
            boxesT = boxes.copy()
            boxesT['x_bottom'] = boxes['x_bottom'] + x * L
            boxesT['y_bottom'] = boxes['y_bottom'] + y * L
            boxesT['z_bottom'] = boxes['z_bottom'] + z * L
            boxesPBC = boxesPBC.append(boxesT)
print(boxesPBC.shape)


(3375, 4)


In [ ]:
# create pairing boxes idx for all 75 boxes, each box has 13 pairing box index
for i in range(num_box_1D**3):
    neiboring_boxes = []
    for j in range(1, 14):
        # if a matching box j can be found for box i, bidx is the absolute index of matching box 
        bidx = boxesPBC.index[(((boxesPBC['x_bottom']- boxes.iloc[i, 0])/4 == d_vector.iloc[j,0]) & 
                ((boxesPBC['y_bottom']- boxes.iloc[i, 1])/4 == d_vector.iloc[j,1]) &
                ((boxesPBC['z_bottom']- boxes.iloc[i, 2])/4 == d_vector.iloc[j,2]))][0]   
        # bidx is the index for the matching box and j is the relative index in the displacement vector
        neiboring_boxes.append([bidx, j])     
    boxes.loc[i, 'pairing_boxes_id'] = neiboring_boxes
boxes.index.name = 'box_id'
boxes.head()

,x_bottom,y_bottom,z_bottom,pairing_boxes_id
box_id,,,,
0,0,0,0,"[[5, 1], [6, 2], [1, 3], [21, 4], [34, 5], [29..."
1,4,0,0,"[[6, 1], [7, 2], [2, 3], [22, 4], [30, 5], [25..."
2,8,0,0,"[[7, 1], [8, 2], [3, 3], [23, 4], [31, 5], [26..."
3,12,0,0,"[[8, 1], [9, 2], [4, 3], [24, 4], [32, 5], [27..."
4,16,0,0,"[[9, 1], [5, 2], [0, 3], [20, 4], [33, 5], [28..."


In [ ]:
def getboxid(point):
    # find the box_id for particles
    # point[:,0] = x , point[:,1] = y, point[:,2] = z  
    box_id = ((num_box_1D**2 * np.floor(point[:,2]/Rcl)) +
             (num_box_1D * np.floor(point[:,1]/Rcl)) +
             (np.floor(point[:,0]/Rcl)))
    box_id = box_id.astype(int)
    return box_id

In [ ]:
np.random.seed(2) 

In [ ]:
def linkedlistleapfrog(N, Rcl, Rc, num_box_1D, steps, dt = 0.0001, eps=1, sigma = 1, m = 1):

    position = np.empty((N, 3, steps+1))    # store position of N particles
    velocity = np.empty((N, 3, steps+1))    # store velocity of N particles
    
    position[:, :, 0] = np.random.uniform(low = 0, high = L, size = (N, 3)) 
    velocity[:, :, 0] = np.random.uniform(low = -0.1, high = 0.1, size = (N, 3)) 
    
    for t in range(1, steps):   

        # df is the dataframe that is worked on in each time step for calculating Forces
        #  rx, ry, rz are relative positions
        df = pd.DataFrame(columns = ['x', 'y', 'z', 'box_id', 'rx', 'ry','rz', 'Fx', 'Fy', 'Fz'])
        
        # read positions at last time step
        p = position[:, :, t-1]
        # initial velocity at t0-dt/2 (-1,1)
        v = velocity[:, :, t-1]
        vx = v[:,0].reshape(N,)
        vy = v[:,1].reshape(N,)
        vz = v[:,2].reshape(N,)

        df['x'] = p[:,0]
        df['y'] = p[:,1]
        df['z'] = p[:,2]
        df['box_id'] = getboxid(p)
        df = pd.merge(df, boxes, on = ['box_id'])

        df['rx'] = df['x'] - df['x_bottom']
        df['ry'] = df['y'] - df['y_bottom']
        df['rz'] = df['z'] - df['z_bottom']
        df['Fx']= 0
        df['Fy']= 0
        df['Fz']= 0

        # create BoxLL containing a linked list of particles for each box 125*1 at time t
        BoxLL = pd.DataFrame(columns = ['inParticles'])
        for b in range(num_box_1D**3):
            pa_in_box = deque(df[df['box_id'] == b].index.tolist())
            if len(pa_in_box):
                pa_in_box.append(None)
            BoxLL = BoxLL.append({'inParticles': pa_in_box}, ignore_index = True)

        # loop for boxes to calculate distances and forces    
        for b in range(num_box_1D**3):

            ll = BoxLL.iloc[b, 0]   # get particles list for box b

            if len(ll) != 0:         # existing empty list = no particles in this box

                '''within box''' 
                i = 0
                p1 = ll[i]     # head of ll
                while p1:
                    j = i+1
                    p2 = ll[j]
                    while p2:
                        # p1, p2 are index of the two particles and then get p1 p2 positions
                        p1_p = [df.loc[p1, 'rx'], df.loc[p1, 'ry'],df.loc[p1, 'rz']]  # position of p1
                        p2_p = [df.loc[p2, 'rx'], df.loc[p2, 'ry'],df.loc[p2, 'rz']]  # position of p2
                        x12 = p1_p[0] - p2_p[0] 
                        y12 = p1_p[1] - p2_p[1] 
                        z12 = p1_p[2] - p2_p[2] 
                        dis = math.sqrt(x12**2 + y12**2 + z12**2)                     # p1 p2 distance
                        if dis < Rc:
                            Fx = 24 * eps * (2*sigma**12/dis**14 - sigma**6/dis**8) * x12  # calculate forces
                            Fy = 24 * eps * (2*sigma**12/dis**14 - sigma**6/dis**8) * y12  # calculate forces
                            Fz = 24 * eps * (2*sigma**12/dis**14 - sigma**6/dis**8) * z12  # calculate forces
                            df.loc[p1, 'Fx'] = df.loc[p1, 'Fx'] + Fx         # update forces stored in the df for particle1/2
                            df.loc[p1, 'Fy'] = df.loc[p1, 'Fy'] + Fy
                            df.loc[p1, 'Fz'] = df.loc[p1, 'Fz'] + Fz
                            df.loc[p2, 'Fx'] = df.loc[p2, 'Fx'] + (-Fx)
                            df.loc[p2, 'Fy'] = df.loc[p2, 'Fy'] + (-Fy)
                            df.loc[p2, 'Fz'] = df.loc[p2, 'Fz'] + (-Fz)            
                        j += 1
                        p2 = ll[j]
                    i += 1
                    p1 = ll[i]    

                '''between boxes'''
                nbid = boxes.loc[b, 'pairing_boxes_id']          # get neighboring box ids for box b
                for nb in nbid:                                  # for each pairing box nb of b
                    nbll = BoxLL.iloc[nb[0], 0]                     # get particles list for paring box nb[0]
                                                                # nb[1] is the relative box id of nb relative to box b in displacement vector
                    if len(nbll) != 0:         # existing empty list = no particles in this box
                        i = 0
                        p1 = ll[i]     # head of ll

                        while p1:
                            j = 0
                            p2 = nbll[j]
                            while p2:
                                # p1, p2 are index of the two particles and then get p1 p2 positions
                                p1_p = [df.loc[p1, 'rx'], df.loc[p1, 'ry'],df.loc[p1, 'rz']]  # relative position of p1
                                p2_p = [df.loc[p2, 'rx'], df.loc[p2, 'ry'],df.loc[p2, 'rz']]  # relative position of p2
                                x12 = p1_p[0] - p2_p[0] - Rcl * d_vector.loc[nb[1],'xb']    # calculting based on displacement
                                y12 = p1_p[1] - p2_p[1] - Rcl * d_vector.loc[nb[1],'yb']
                                z12 = p1_p[2] - p2_p[2] - Rcl * d_vector.loc[nb[1],'xb']
                                dis = math.sqrt(x12**2 + y12**2 + z12**2)                     # p1 p2 distance
                                if dis < Rc:
                                    Fx = 24 * eps * (2*sigma**12/dis**14 - sigma**6/dis**8) * x12  # calculate forces
                                    Fy = 24 * eps * (2*sigma**12/dis**14 - sigma**6/dis**8) * y12  # calculate forces
                                    Fz = 24 * eps * (2*sigma**12/dis**14 - sigma**6/dis**8) * z12  # calculate forces
                                    df.loc[p1, 'Fx'] = df.loc[p1, 'Fx'] + Fx         # update forces stored in the df for particle1/2
                                    df.loc[p1, 'Fy'] = df.loc[p1, 'Fy'] + Fy
                                    df.loc[p1, 'Fz'] = df.loc[p1, 'Fz'] + Fz
                                    df.loc[p2, 'Fx'] = df.loc[p2, 'Fx'] + (-Fx)
                                    df.loc[p2, 'Fy'] = df.loc[p2, 'Fy'] + (-Fy)
                                    df.loc[p2, 'Fz'] = df.loc[p2, 'Fz'] + (-Fz)            
                                j += 1
                                p2 = nbll[j]
                            i += 1
                            p1 = ll[i]    
        '''
        leapfrag update
        '''
        # velocity for t0+dt/2
        vx += dt * (df['Fx']/m)    
        vy += dt * (df['Fy']/m) 
        vz += dt * (df['Fz']/m) 

        # position for t0 + dt
        tx = df['x'] + ((vx * dt))
        ty = df['y'] + ((vy * dt))
        tz = df['z'] + ((vz * dt))

        tx = np.where(tx >= num_box_1D * Rcl, tx-num_box_1D * Rcl, tx)
        ty = np.where(ty >= num_box_1D * Rcl, ty-num_box_1D * Rcl, ty)  
        tz = np.where(tz >= num_box_1D * Rcl, tz-num_box_1D * Rcl, tz)

        tx = np.where(tx < 0, tx+num_box_1D * Rcl, tx)
        ty = np.where(ty < 0, ty+num_box_1D * Rcl, ty)  
        tz = np.where(tz < 0, tz+num_box_1D * Rcl, tz)


        df['x'] = tx
        df['y'] = ty
        df['z'] = tz

        position[:, 0, t] = tx
        position[:, 1, t] = ty
        position[:, 2, t] = tz

        velocity[:, 0, t] = vx
        velocity[:, 1, t] = vy
        velocity[:, 2, t] = vz

    return position, velocity

In [ ]:
positions, velocities = linkedlistleapfrog(N = 100, Rcl = 4, Rc = 2.5, num_box_1D = 5, steps = 100, dt = 0.0001)

In [ ]:
def draw(position, finvervals = 300, divide = 2):
    xyz = position
    graphtitle = r'Linked List ' + 'timestep={}'
    
    fig = plt.figure(figsize = (8, 8))
    ax = p3.Axes3D(fig)
    title = ax.set_title('Linked List ')

    #axislimit = 25
    # Setting the axes properties
    ax.set_xlim3d([-5, 25])
    ax.set_xlabel('X')

    ax.set_ylim3d([-5, 25])
    ax.set_ylabel('Y')

    ax.set_zlim3d([-5, 25])
    ax.set_zlabel('Z')
    #C = np.array([[255, 0, 0], [0,0, 255]])
    
    
    # Provide starting angle for the view.
    ax.view_init(15, 45)

    title.set_text('Linked List')
    # create and display boundaries
    #limit = 10
    x = np.linspace(0, 20, 100)
    y1 = np.array([20]*100)
    y2 = np.array([0]*100)
    z1 = np.array([20]*100)
    z2 = np.array([0]*100)
    x1 = np.array([20]*100)
    x2 = np.array([0]*100)
    y = np.linspace(0, 20, 100)    
    z = np.linspace(0, 20, 100) 
    
    ax.grid(True)

    # Hide axes ticks
    #ax.set_xticks([])
    #ax.set_yticks([])
    #ax.set_zticks([])


    def update_graph(num):

        graph._offsets3d = (xyz[:, 0, num*divide], xyz[:, 1, num*divide], xyz[:, 2, num*divide])
        title.set_text(graphtitle.format(num*divide))

    ax.plot(x, y1, z1, c = 'r')
    ax.plot(x, y2, z1, c = 'r')
    ax.plot(x, y1, z2, c = 'r')
    ax.plot(x, y2, z2, c = 'r')
    ax.plot(x1, y, z1, c = 'r')
    ax.plot(x1, y, z2, c = 'r')
    ax.plot(x2, y, z1, c = 'r')
    ax.plot(x2, y, z2, c = 'r')
    ax.plot(x1, y1, z, c = 'r')
    ax.plot(x1, y2, z, c = 'r')
    ax.plot(x2, y1, z, c = 'r')
    ax.plot(x2, y2, z, c = 'r')
    graph = ax.scatter(xyz[:, 0, 0], xyz[:, 1, 0], xyz[:, 2, 0])
    
    timesteps = int(position.shape[2]/divide)
    ani = animation.FuncAnimation(fig, update_graph, timesteps, 
                                   interval=finvervals, blit=False)


    video = ani.to_html5_video()
    html = display.HTML(video)
    display.display(html)
    plt.close()
    plt.show()

In [ ]:
draw(positions, finvervals = 300, divide = 2)